In [2]:
import pandas as pd
import numpy as np
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_row", 500)
pd.set_option('display.max.columns', None)
pd.set_option('display.precision', 2)
pd.options.display.float_format = '${:,.2f}'.format
from datetime import date, time, datetime, timedelta
import dateutil
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt

## Nyack CBIZ report import / cleanup

In [3]:
nyack_cbiz_report = pd.read_csv(r"C:\Users\Michael\OneDrive\Documents\Work\conversion\SIT3\sit3_environment\fixed_assets\source_files_2021\nyack\Montefiore Nyack Hospital-Revalued 2021 Masterfile_D.csv",
                         
                          usecols=['Asset Display Id', 'Asset Activity Code', 'Asset Tag Number',
                                   'Account', 'Asset Class', 'Department Number',
                                   'Department', 'Site Name', 'Building Name', 'Quantity', 'Asset Description',
                                    'Acquisition Date','Historical Cost', 'Life Months', 'Period Depreciation',
                                   'Accumulated Depreciation', 'Net Book Value'], )



In [8]:
nyack_cbiz_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17301 entries, 0 to 17300
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Asset Display Id          17301 non-null  int64  
 1   Asset Activity Code       17301 non-null  object 
 2   Asset Tag Number          17301 non-null  object 
 3   Account                   17301 non-null  object 
 4   Asset Class               17301 non-null  object 
 5   Department Number         17279 non-null  float64
 6   Department                17279 non-null  object 
 7   Site Name                 17301 non-null  object 
 8   Building Name             17301 non-null  object 
 9   Quantity                  17301 non-null  int64  
 10  Asset Description         17299 non-null  object 
 11  Acquisition Date          17301 non-null  object 
 12  Historical Cost           17301 non-null  object 
 13  Life Months               17046 non-null  float64
 14  Period

In [9]:
# sum all the missing values in the df by applying df.isnull() and passing the
# resulting boolean series into np.count_nonzero() method
np.count_nonzero(nyack_cbiz_report.isnull())

301

In [10]:
nyack_cbiz_report.iloc[14374]

Asset Display Id                                              14382
Asset Activity Code                                          Active
Asset Tag Number                                           21414382
Account                                              MAJOR MOVEABLE
Asset Class                                       MEDICAL EQUIPMENT
Department Number                                     $1,500,684.00
Department                                     BREAST CANCER CENTER
Site Name                                            NYACK HOSPITAL
Building Name                                        NYACK HOSPITAL
Quantity                                                          1
Asset Description           INVIVO CANCELLATION    / VOUCHER CREDIT
Acquisition Date                                           8/1/2014
Historical Cost                                          -79,000.00
Life Months                                                  $72.00
Period Depreciation                             

In [11]:
# calculate the missing fields by column
num_rows = nyack_cbiz_report.shape[0]
num_missing = num_rows - nyack_cbiz_report.count()
num_missing

Asset Display Id              0
Asset Activity Code           0
Asset Tag Number              0
Account                       0
Asset Class                   0
Department Number            22
Department                   22
Site Name                     0
Building Name                 0
Quantity                      0
Asset Description             2
Acquisition Date              0
Historical Cost               0
Life Months                 255
Period Depreciation           0
Accumulated Depreciation      0
Net Book Value                0
dtype: int64

In [8]:
# mmc_tb = pd.read_csv(r'C:\Users\Michael\OneDrive\Documents\Work\conversion\august_2021\sap_tb_2020.csv')

In [9]:
nyack_cbiz_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16423 entries, 0 to 16422
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Asset Display Id          16423 non-null  int64  
 1   Asset Activity Code       16423 non-null  object 
 2   Asset Tag Number          16423 non-null  object 
 3   Account                   16423 non-null  object 
 4   Asset Class               16423 non-null  object 
 5   Department Number         15262 non-null  object 
 6   Department                14880 non-null  object 
 7   Site Name                 16423 non-null  object 
 8   Building Name             16417 non-null  object 
 9   Quantity                  16423 non-null  int64  
 10  Asset Description         16421 non-null  object 
 11  Acquisition Date          16423 non-null  object 
 12  Historical Cost           16423 non-null  object 
 13  Life Months               16168 non-null  float64
 14  Period

In [12]:
nyack_cbiz_report.columns

Index(['Asset Display Id', 'Asset Activity Code', 'Asset Tag Number',
       'Account', 'Asset Class', 'Department Number', 'Department',
       'Site Name', 'Building Name', 'Quantity', 'Asset Description',
       'Acquisition Date', 'Historical Cost', 'Life Months',
       'Period Depreciation', 'Accumulated Depreciation', 'Net Book Value'],
      dtype='object')

In [13]:
nyack_cbiz_report['Life Months'] = nyack_cbiz_report['Life Months'].fillna(0)

In [14]:
nyack_cbiz_report[nyack_cbiz_report['Life Months'].isnull()]

,Asset Display Id,Asset Activity Code,Asset Tag Number,Account,Asset Class,Department Number,Department,Site Name,Building Name,Quantity,Asset Description,Acquisition Date,Historical Cost,Life Months,Period Depreciation,Accumulated Depreciation,Net Book Value


In [15]:
nyack_cbiz_report['Life Months'] = nyack_cbiz_report['Life Months'].astype('int32')

In [16]:
nyack_cbiz_report['Acquisition Date'] = pd.to_datetime(nyack_cbiz_report['Acquisition Date'])



In [17]:
nyack_cbiz_report[nyack_cbiz_report['Historical Cost'].str.contains('NaN')]

,Asset Display Id,Asset Activity Code,Asset Tag Number,Account,Asset Class,Department Number,Department,Site Name,Building Name,Quantity,Asset Description,Acquisition Date,Historical Cost,Life Months,Period Depreciation,Accumulated Depreciation,Net Book Value


In [16]:
# nyack_cbiz_report['Historical Cost'] = nyack_cbiz_report['Historical Cost'].astype('str')

In [17]:
# nyack_cbiz_report['Period Depreciation'] = nyack_cbiz_report['Period Depreciation'].astype('str')

In [18]:
def ser_to_float(ser):
    ser = ser.str.strip()
#     ser = ser.apply(lambda x: x.replace(' - ', ''))
#     ser = ser.apply(lambda x: x.replace('-', ''))
    ser = ser.apply(lambda x: x.replace(',', ''))
#     ser = ser.apply(lambda x: x.replace('-', ''))
    ser = ser.apply(lambda x: x.replace(' ', ''))
    ser = ser.apply(lambda x: x.replace('(', ''))
    ser = ser.apply(lambda x: x.replace(')', ''))
    ser = ser.apply(lambda x: x.replace('Nan', ''))
    ser = ser.astype(float)
    return ser


    

In [19]:
nyack_cbiz_report['Historical Cost'] = ser_to_float(nyack_cbiz_report['Historical Cost'])
nyack_cbiz_report['Period Depreciation'] = ser_to_float(nyack_cbiz_report['Period Depreciation'])
nyack_cbiz_report['Accumulated Depreciation'] = ser_to_float(nyack_cbiz_report['Accumulated Depreciation'])
nyack_cbiz_report['Net Book Value'] = ser_to_float(nyack_cbiz_report['Net Book Value'])

In [20]:
nyack_cbiz_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17301 entries, 0 to 17300
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Asset Display Id          17301 non-null  int64         
 1   Asset Activity Code       17301 non-null  object        
 2   Asset Tag Number          17301 non-null  object        
 3   Account                   17301 non-null  object        
 4   Asset Class               17301 non-null  object        
 5   Department Number         17279 non-null  float64       
 6   Department                17279 non-null  object        
 7   Site Name                 17301 non-null  object        
 8   Building Name             17301 non-null  object        
 9   Quantity                  17301 non-null  int64         
 10  Asset Description         17299 non-null  object        
 11  Acquisition Date          17301 non-null  datetime64[ns]
 12  Historical Cost   

In [21]:
nyack_cbiz_report.tail()

,Asset Display Id,Asset Activity Code,Asset Tag Number,Account,Asset Class,Department Number,Department,Site Name,Building Name,Quantity,Asset Description,Acquisition Date,Historical Cost,Life Months,Period Depreciation,Accumulated Depreciation,Net Book Value
17296,21758,New,20211278,CAPITAL - MEDICAL VILLAGE EQUIPMENT,MEDICAL EQUIPMENT,"$1,500,684.00",BREAST CANCER CENTER,NYACK HOSPITAL,NYACK HOSPITAL,1,WARMING CABINETS-50% DEPOSIT,2020-06-01,"$38,476.26",240,"$1,923.81","$2,885.72","$35,590.54"
17297,21759,New,20211279,CAPITAL - MEDICAL VILLAGE CONSTRUCTION,BUILDINGS,"$1,500,570.00",EMERGENCY DEPT GENERAL,NYACK HOSPITAL,NYACK HOSPITAL,1,DAMAGE WAIVER - 4X4 RENTAL BOOTH - PER MONTH\...,2020-06-01,$8.00,60,$1.60,$2.40,$5.60
17298,21760,New,20211280,CAPITAL - OTHER COMPUTER EQUIPMENT,COMPUTER EQUIPMENT,"$1,500,017.00",TECHNOLOGY SERVICES,NYACK HOSPITAL,NYACK HOSPITAL,1,INV# 0071983 TRAVEL EXPENSES,2020-06-01,"$7,000.00",48,"$1,750.00","$2,625.00","$4,375.00"
17299,21761,New,20211281,CAPITAL - OTHER COMPUTER EQUIPMENT,COMPUTER EQUIPMENT,"$1,500,017.00",TECHNOLOGY SERVICES,NYACK HOSPITAL,NYACK HOSPITAL,1,INV# 0071983 TRAVEL EXPENSES,2020-06-01,"$14,000.00",48,"$3,500.00","$5,250.00","$8,750.00"
17300,21762,New,20211282,CAPITAL - OTHER COMPUTER EQUIPMENT,COMPUTER EQUIPMENT,"$1,500,017.00",TECHNOLOGY SERVICES,NYACK HOSPITAL,NYACK HOSPITAL,1,INV# 7002015000 MEDICATION RECONCILIATION,2020-06-01,$240.00,48,$60.00,$90.00,$150.00


In [22]:
nyack_cbiz_report[['Historical Cost', 'Accumulated Depreciation', 'Net Book Value']].sum()

Historical Cost            $172,236,795.28
Accumulated Depreciation    $63,291,098.44
Net Book Value             $108,945,696.84
dtype: float64

### Nyack asset book

In [23]:
# Load the member entity asset_book transformation file
me_load_asset_book = pd.read_csv(r'C:\Users\mikra\OneDrive\Documents\Work\conversion\SIT3\november_AX2\fixed_assets\Entity_Asset_Book_Import.csv',
                                usecols=['FinanceEnterpriseGroup', 'AssetImport', 'Book', 'RunGroup', 'Active', 'Life',
                                         'LifeRemaining', 'Basis', 'Method', 'InServiceDate', 'Convention',
                                         'YearToDateDepreciation', 'LifeToDateDepreciation'])

In [24]:
# me_load_asset_item = pd.read_csv(r'C:\Users\mikra\OneDrive\Documents\Work\conversion\SIT3\november_AX2\fixed_assets\Entity_Asset_Item_Import.csv')

In [25]:

me_load_asset_book_groups = me_load_asset_book.groupby('RunGroup')

In [26]:
nyack_load_asset_book = me_load_asset_book_groups.get_group('NYK_ASSET')

In [27]:
nyack_load_asset_book.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10949 entries, 94085 to 105033
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   FinanceEnterpriseGroup  10949 non-null  int64  
 1   AssetImport             10949 non-null  object 
 2   Book                    10949 non-null  object 
 3   RunGroup                10949 non-null  object 
 4   Active                  10949 non-null  bool   
 5   Life                    10949 non-null  int64  
 6   LifeRemaining           10949 non-null  int64  
 7   Basis                   10949 non-null  float64
 8   Method                  10491 non-null  object 
 9   InServiceDate           10949 non-null  int64  
 10  Convention              10949 non-null  object 
 11  YearToDateDepreciation  0 non-null      float64
 12  LifeToDateDepreciation  10949 non-null  float64
dtypes: bool(1), float64(3), int64(4), object(5)
memory usage: 1.1+ MB


In [28]:
nyack_load_asset_book['Book'].unique()

array(['GL'], dtype=object)

In [29]:
nyack_load_asset_book['RunGroup'].unique()

array(['NYK_ASSET'], dtype=object)

In [30]:
# nyack_load_asset_book['Life'] = nyack_load_asset_book['Life'].fillna(0)
nyack_load_asset_book[nyack_load_asset_book['Life'].isnull()]

,FinanceEnterpriseGroup,AssetImport,Book,RunGroup,Active,Life,LifeRemaining,Basis,Method,InServiceDate,Convention,YearToDateDepreciation,LifeToDateDepreciation


In [31]:
# nyack_load_asset_book['LifeRemaining'] = nyack_load_asset_book['LifeRemaining'].fillna(0)
nyack_load_asset_book[nyack_load_asset_book['LifeRemaining'].isnull()]

,FinanceEnterpriseGroup,AssetImport,Book,RunGroup,Active,Life,LifeRemaining,Basis,Method,InServiceDate,Convention,YearToDateDepreciation,LifeToDateDepreciation


In [32]:
nyack_load_asset_book['Life'] = nyack_load_asset_book['Life'].astype('int')
nyack_load_asset_book['LifeRemaining'] = nyack_load_asset_book['LifeRemaining'].astype('int')

<ipython-input-32-556655e4aff6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyack_load_asset_book['Life'] = nyack_load_asset_book['Life'].astype('int')
<ipython-input-32-556655e4aff6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyack_load_asset_book['LifeRemaining'] = nyack_load_asset_book['LifeRemaining'].astype('int')


In [33]:
nyack_load_asset_book['cbiz_index'] = nyack_load_asset_book['AssetImport'].apply(lambda x: int(x.split('_')[1]))

<ipython-input-33-6003adef4b7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyack_load_asset_book['cbiz_index'] = nyack_load_asset_book['AssetImport'].apply(lambda x: int(x.split('_')[1]))


In [34]:
nyack_load_asset_book['cbiz_index'] = nyack_load_asset_book['cbiz_index'].astype('int')

<ipython-input-34-3f91ab87f037>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyack_load_asset_book['cbiz_index'] = nyack_load_asset_book['cbiz_index'].astype('int')


In [35]:
nyack_load_asset_book.tail()

,FinanceEnterpriseGroup,AssetImport,Book,RunGroup,Active,Life,LifeRemaining,Basis,Method,InServiceDate,Convention,YearToDateDepreciation,LifeToDateDepreciation,cbiz_index
105029,1,NYK_20476,GL,NYK_ASSET,True,84,72,"$234,738.04",SL,20200108,HY,NaN,"$16,767.00",20476
105030,1,NYK_20477,GL,NYK_ASSET,True,84,72,"$39,000.00",SL,20200114,HY,NaN,"$2,785.71",20477
105031,1,NYK_20478,GL,NYK_ASSET,True,84,78,"$91,000.00",SL,20200701,HY,NaN,"$6,500.00",20478
105032,1,NYK_20479,GL,NYK_ASSET,True,60,47,"$116,640.40",SL,20191201,HY,NaN,"$34,992.12",20479
105033,1,NYK_20480,GL,NYK_ASSET,True,240,227,"$231,830.00",SL,20191201,HY,NaN,"$17,387.25",20480


In [36]:
nyack_load_asset_book['InServiceDate'] = nyack_load_asset_book['InServiceDate'].astype('str')

<ipython-input-36-ee7ebc1c57af>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyack_load_asset_book['InServiceDate'] = nyack_load_asset_book['InServiceDate'].astype('str')


In [37]:
# create a new column in me_load_asset_book that contains the 'InServiceDate' elements to datetime objects

nyack_load_asset_book['InServiceDate_dt'] = pd.to_datetime(nyack_load_asset_book['InServiceDate'])


<ipython-input-37-c18860c2a496>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyack_load_asset_book['InServiceDate_dt'] = pd.to_datetime(nyack_load_asset_book['InServiceDate'])


In [38]:
nyack_load_asset_book.Basis.sum()

142561523.94

In [39]:
nyack_load_asset_book.head()

,FinanceEnterpriseGroup,AssetImport,Book,RunGroup,Active,Life,LifeRemaining,Basis,Method,InServiceDate,Convention,YearToDateDepreciation,LifeToDateDepreciation,cbiz_index,InServiceDate_dt
94085,1,NYK_87,GL,NYK_ASSET,True,12,0,$10.00,SL,20140801,HY,NaN,$10.00,87,2014-08-01
94086,1,NYK_88,GL,NYK_ASSET,True,12,0,$10.00,SL,20140801,HY,NaN,$10.00,88,2014-08-01
94087,1,NYK_89,GL,NYK_ASSET,True,12,0,$20.00,SL,20140801,HY,NaN,$20.00,89,2014-08-01
94088,1,NYK_90,GL,NYK_ASSET,True,12,0,$20.00,SL,20140801,HY,NaN,$20.00,90,2014-08-01
94089,1,NYK_91,GL,NYK_ASSET,True,12,0,$20.00,SL,20140801,HY,NaN,$20.00,91,2014-08-01


In [40]:
nyack_load_asset_book.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10949 entries, 94085 to 105033
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   FinanceEnterpriseGroup  10949 non-null  int64         
 1   AssetImport             10949 non-null  object        
 2   Book                    10949 non-null  object        
 3   RunGroup                10949 non-null  object        
 4   Active                  10949 non-null  bool          
 5   Life                    10949 non-null  int32         
 6   LifeRemaining           10949 non-null  int32         
 7   Basis                   10949 non-null  float64       
 8   Method                  10491 non-null  object        
 9   InServiceDate           10949 non-null  object        
 10  Convention              10949 non-null  object        
 11  YearToDateDepreciation  0 non-null      float64       
 12  LifeToDateDepreciation  10949 non-null  f

### nyack assets file load

In [41]:
me_load_asset = pd.read_csv(r'C:\Users\mikra\OneDrive\Documents\Work\conversion\SIT3\november_AX2\fixed_assets\Entity_Asset_Import.csv',
                           usecols=['FinanceEnterpriseGroup', 'AssetImport', 'RunGroup','Description', 'Classification',
                                   'TagNumber','AssetGroup', 'AssetCategory', 'Reference2',
                                   'Company','AssetType.Type','AssetAccountingUnitGroup','AssetAccountGroup',])

C:\Users\mikra\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [42]:
me_load_asset.columns

Index(['FinanceEnterpriseGroup', 'AssetImport', 'RunGroup', 'Description',
       'Classification', 'TagNumber', 'AssetGroup', 'AssetCategory',
       'Reference2', 'Company', 'AssetType.Type', 'AssetAccountingUnitGroup',
       'AssetAccountGroup'],
      dtype='object')

In [43]:
me_load_asset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105034 entries, 0 to 105033
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   FinanceEnterpriseGroup    105034 non-null  int64 
 1   AssetImport               105034 non-null  object
 2   RunGroup                  105034 non-null  object
 3   Description               105032 non-null  object
 4   Classification            105034 non-null  object
 5   TagNumber                 95502 non-null   object
 6   AssetGroup                105034 non-null  object
 7   AssetCategory             105034 non-null  object
 8   Reference2                104494 non-null  object
 9   Company                   105034 non-null  int64 
 10  AssetType.Type            105034 non-null  object
 11  AssetAccountingUnitGroup  105034 non-null  int64 
 12  AssetAccountGroup         105034 non-null  object
dtypes: int64(3), object(10)
memory usage: 10.4+ MB


In [44]:
me_load_asset['RunGroup'].unique()

array(['BURK_REHAB_ASSET', 'WPH_ASSET', 'SLH_ASSET', 'BURKE_COVID_ASSET',
       'BURKE_FOUNDATION_ASSET', 'NYK_ASSET'], dtype=object)

In [45]:
me_load_asset_groups = me_load_asset.groupby('RunGroup')

In [46]:
nyack_asset_load = me_load_asset_groups.get_group('NYK_ASSET')

In [47]:
# nyack_asset_load = pd.read_csv(r"C:\Users\mikra\OneDrive\Documents\Work\conversion\oct_2021_ax2\fixed_assets\nyack\NYK_AssetImport_Load.csv",
#                             usecols=(['FinanceEnterpriseGroup', 'AssetImport', 'Description',
#        'Classification', 'TagNumber', 'AssetGroup', 'AssetCategory',
#        'Reference2', 'Company', 'Currency', 'AssetLocation',
#        'AssetAccountingUnitGroup', 'AssetAccountGroup']))

In [48]:
nyack_asset_load.columns

Index(['FinanceEnterpriseGroup', 'AssetImport', 'RunGroup', 'Description',
       'Classification', 'TagNumber', 'AssetGroup', 'AssetCategory',
       'Reference2', 'Company', 'AssetType.Type', 'AssetAccountingUnitGroup',
       'AssetAccountGroup'],
      dtype='object')

In [49]:
nyack_asset_load.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10949 entries, 94085 to 105033
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   FinanceEnterpriseGroup    10949 non-null  int64 
 1   AssetImport               10949 non-null  object
 2   RunGroup                  10949 non-null  object
 3   Description               10947 non-null  object
 4   Classification            10949 non-null  object
 5   TagNumber                 10949 non-null  object
 6   AssetGroup                10949 non-null  object
 7   AssetCategory             10949 non-null  object
 8   Reference2                10409 non-null  object
 9   Company                   10949 non-null  int64 
 10  AssetType.Type            10949 non-null  object
 11  AssetAccountingUnitGroup  10949 non-null  int64 
 12  AssetAccountGroup         10949 non-null  object
dtypes: int64(3), object(10)
memory usage: 1.2+ MB


In [50]:
nyack_asset_load['AssetImport'].nunique()

10949

In [51]:
nyack_asset_load['cbiz_index'] = nyack_asset_load['AssetImport'].apply(lambda x: int(x.split('_')[1]))
# nyack_asset_load['cbiz_asset_ids'] = nyack_asset_load['AssetImport'].apply(lambda x: (x.split('_')[1]))

<ipython-input-51-f3fd18f8f22d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyack_asset_load['cbiz_index'] = nyack_asset_load['AssetImport'].apply(lambda x: int(x.split('_')[1]))


In [52]:
nyack_asset_load['Company'].unique()

array([2300], dtype=int64)

In [53]:
nyack_asset_load

,FinanceEnterpriseGroup,AssetImport,RunGroup,Description,Classification,TagNumber,AssetGroup,AssetCategory,Reference2,Company,AssetType.Type,AssetAccountingUnitGroup,AssetAccountGroup,cbiz_index
94085,1,NYK_1,NYK_ASSET,COMPUTER UPGRADE 64MB TO 128MB,P,1248,CONVERSION,D,LTHHC,2300,MAJ_MOV,1,CIP,1
94086,1,NYK_10,NYK_ASSET,CRYO THERM 3,P,00029,CONVERSION,D,BLOOD BANK,2300,MED_EQUIP,1,CIP,10
94087,1,NYK_100,NYK_ASSET,CHAIR ARM SWIVEL MTL UPH,P,00231,CONVERSION,D,LABORATORY,2300,MAJ_MOV,1,CIP,100
94088,1,NYK_10000,NYK_ASSET,INFUSION SYS AUTOMATED PRESSURE,P,96000165,CONVERSION,D,OPERATING ROOM,2300,MED_EQUIP,1,CIP,10000
94089,1,NYK_10001,NYK_ASSET,COST ACCTG,P,96000166,CONVERSION,D,ACCOUNTING,2300,MAJ_MOV,1,CIP,10001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105029,1,NYK_9995,NYK_ASSET,SOFTWARE H/C INFO SYS,P,96000160,CONVERSION,D,LTHHC,2300,SFTWRE,1,CIP,9995
105030,1,NYK_9996,NYK_ASSET,PUMP CHEMICAL W/ACC,P,96000161,CONVERSION,D,MAINTENANCE,2300,MED_EQUIP,1,CIP,9996
105031,1,NYK_9997,NYK_ASSET,PROCESSOR SYS I,P,96000162,CONVERSION,D,OPERATING ROOM,2300,MED_EQUIP,1,CIP,9997
105032,1,NYK_9998,NYK_ASSET,CABINETS BLUE PRINT,P,96000163,CONVERSION,D,MAINTENANCE,2300,MAJ_MOV,1,CIP,9998


In [54]:
nyack_cbiz_report.columns

Index(['Asset Display Id', 'Asset Activity Code', 'Asset Tag Number',
       'Account', 'Asset Class', 'Department Number', 'Department',
       'Site Name', 'Building Name', 'Quantity', 'Asset Description',
       'Acquisition Date', 'Historical Cost', 'Life Months',
       'Period Depreciation', 'Accumulated Depreciation', 'Net Book Value'],
      dtype='object')

In [55]:
nyack_cbiz_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16423 entries, 0 to 16422
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Asset Display Id          16423 non-null  int64         
 1   Asset Activity Code       16423 non-null  object        
 2   Asset Tag Number          16423 non-null  object        
 3   Account                   16423 non-null  object        
 4   Asset Class               16423 non-null  object        
 5   Department Number         15262 non-null  object        
 6   Department                14880 non-null  object        
 7   Site Name                 16423 non-null  object        
 8   Building Name             16417 non-null  object        
 9   Quantity                  16423 non-null  int64         
 10  Asset Description         16421 non-null  object        
 11  Acquisition Date          16423 non-null  datetime64[ns]
 12  Historical Cost   

In [56]:
nyack_asset_load.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10949 entries, 94085 to 105033
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   FinanceEnterpriseGroup    10949 non-null  int64 
 1   AssetImport               10949 non-null  object
 2   RunGroup                  10949 non-null  object
 3   Description               10947 non-null  object
 4   Classification            10949 non-null  object
 5   TagNumber                 10949 non-null  object
 6   AssetGroup                10949 non-null  object
 7   AssetCategory             10949 non-null  object
 8   Reference2                10409 non-null  object
 9   Company                   10949 non-null  int64 
 10  AssetType.Type            10949 non-null  object
 11  AssetAccountingUnitGroup  10949 non-null  int64 
 12  AssetAccountGroup         10949 non-null  object
 13  cbiz_index                10949 non-null  int64 
dtypes: int64(4), obje

In [57]:
nyack_load_asset_book.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10949 entries, 94085 to 105033
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   FinanceEnterpriseGroup  10949 non-null  int64         
 1   AssetImport             10949 non-null  object        
 2   Book                    10949 non-null  object        
 3   RunGroup                10949 non-null  object        
 4   Active                  10949 non-null  bool          
 5   Life                    10949 non-null  int32         
 6   LifeRemaining           10949 non-null  int32         
 7   Basis                   10949 non-null  float64       
 8   Method                  10491 non-null  object        
 9   InServiceDate           10949 non-null  object        
 10  Convention              10949 non-null  object        
 11  YearToDateDepreciation  0 non-null      float64       
 12  LifeToDateDepreciation  10949 non-null  f


#### Need to tie these numbers back to the Infor trial balances

In [58]:
cbiz_2300_merge = nyack_asset_load.merge(nyack_cbiz_report,how='inner',
left_on='cbiz_index', right_on='Asset Display Id')

cbiz_2300_merge[['Historical Cost', 'Accumulated Depreciation', 'Net Book Value']].sum()

Historical Cost            $142,561,523.94
Accumulated Depreciation    $50,731,522.47
Net Book Value              $91,830,001.47
dtype: float64

In [59]:
cbiz_2300_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10949 entries, 0 to 10948
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   FinanceEnterpriseGroup    10949 non-null  int64         
 1   AssetImport               10949 non-null  object        
 2   RunGroup                  10949 non-null  object        
 3   Description               10947 non-null  object        
 4   Classification            10949 non-null  object        
 5   TagNumber                 10949 non-null  object        
 6   AssetGroup                10949 non-null  object        
 7   AssetCategory             10949 non-null  object        
 8   Reference2                10409 non-null  object        
 9   Company                   10949 non-null  int64         
 10  AssetType.Type            10949 non-null  object        
 11  AssetAccountingUnitGroup  10949 non-null  int64         
 12  AssetAccountGroup 

In [60]:
cbiz_asset_and_book_2300_merge = cbiz_2300_merge.merge(nyack_load_asset_book, how='inner',
                                 left_on='cbiz_index', right_on='cbiz_index').set_index('cbiz_index').\
                                 drop(['FinanceEnterpriseGroup_x', 'AssetImport_y', 
                                 'Classification',  'TagNumber', 'AssetGroup',
                                 'AssetAccountingUnitGroup', 'AssetAccountGroup', 'Active', 'Reference2',
                                 'AssetAccountingUnitGroup', 'FinanceEnterpriseGroup_y'], 
                                 axis=1)

cbiz_asset_and_book_2300_merge[['Historical Cost', 'Basis', 'Accumulated Depreciation', 
                                'LifeToDateDepreciation','Net Book Value']].sum()

Historical Cost            $142,561,523.94
Basis                      $142,561,523.94
Accumulated Depreciation    $50,731,522.47
LifeToDateDepreciation      $50,731,522.47
Net Book Value              $91,830,001.47
dtype: float64

In [ ]:
# nyack_cost_basis_diff = cbiz_asset_and_book_2300_merge[cbiz_asset_and_book_2300_merge['Historical Cost'] != cbiz_asset_and_book_2300_merge['Basis']]
# (nyack_cost_basis_diff['Historical Cost'] - nyack_cost_basis_diff['Basis']).sum()

In [ ]:
# nyack_cost_basis_diff

In [ ]:
# (nyack_cost_basis_diff['Historical Cost'] - nyack_cost_basis_diff['Basis']).sum()

In [ ]:
# (nyack_cost_basis_diff['Accumulated Depreciation'] - nyack_cost_basis_diff['LifeToDateDepreciation']).sum()

In [ ]:
cbiz_asset_and_book_2300_merge.tail()

In [ ]:
cbiz_asset_and_book_2300_merge.shape

In [ ]:
cbiz_asset_and_book_2300_merge['AssetCategory'].value_counts()

### Nyack Asset Visualizations

In [ ]:
cbiz_asset_and_book_2300_merge.columns

In [ ]:
cbiz_asset_and_book_2300_merge.info()

In [ ]:
cbiz_asset_and_book_2300_merge['Asset Class'].unique()

In [ ]:
nyack_asset_class_groups = cbiz_asset_and_book_2300_merge.groupby('Asset Class')

In [ ]:
nyack_medical_equipment = nyack_asset_class_groups.get_group('MEDICAL EQUIPMENT')

In [ ]:
# group the medical equipment asset group by inservice date and sum the historical cost by year
nyack_medical_equipment.groupby(pd.Grouper(key='InServiceDate_dt', axis=0, freq='Y'))['Historical Cost'].sum()

### Load the FSM asset register for all companies

In [ ]:
asset_download_all_entities = pd.read_csv(r"C:\Users\mikra\OneDrive\Documents\Work\conversion\SIT3\november_AX2\fixed_assets\Asset Download All Entities 113021.csv")

In [ ]:
# asset_download_all_entities.info()

In [ ]:
# look for NaN values in the company column

asset_download_all_entities[asset_download_all_entities['Company'].isna()]


In [ ]:
# replace 'NaN' in company column for a record

# asset_download_all_entities.loc[126381, 'Company'] = ''

In [ ]:
# cast the company column as a string and strip out the excess digits

asset_download_all_entities['Company'] = asset_download_all_entities['Company'].astype('str').apply(lambda x: x[:4])

In [ ]:
# group the FSM asset register by company

fsm_asset_register_groupby_company = asset_download_all_entities.groupby('Company')


In [ ]:
# look at the asset register company groups keys

fsm_asset_register_groupby_company.groups.keys()

### Find Nyack 2300 records that are missing from the FSM asset register

In [ ]:
assets_group_2300 = fsm_asset_register_groupby_company.get_group('2300')

In [ ]:
assets_group_2300.columns

In [ ]:
cbiz_asset_and_book_2300_merge.columns

In [ ]:
cbiz_asset_and_book_2300_merge['AssetImport_x'] = cbiz_asset_and_book_2300_merge['AssetImport_x'].apply(lambda x: x.strip())
assets_group_2300['Reference1'] = assets_group_2300['Reference1'].apply(lambda x: x.strip())

In [ ]:
assets_group_2300.head()

In [ ]:
assets_group_2300[['ItemTotalBaseCost', 'ItemTotalTransactionCost', 'TotalItemsBaseCost',
                   'PostingBookBasis','PostingBookBookValue', 'TotalAccumulatedDepreciation', ]].sum()

In [ ]:
assets_group_2300.info()

In [ ]:
assets_group_2300['Reference1'] = assets_group_2300['Reference1'].apply(lambda x: x.strip())

In [ ]:
# assets_group_2300['ItemTotalBaseCost'] = assets_group_2300['ItemTotalBaseCost'].apply(lambda x: float(x))

In [ ]:
assets_group_2300.head()

In [ ]:
assets_group_2300.info()

In [ ]:
# add ther merge index column to the company asset group

assets_group_2300['merge_index'] = assets_group_2300['Reference1'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
assets_group_2300 = assets_group_2300.set_index('merge_index')

In [ ]:
assets_group_2300['cbiz_index'] = assets_group_2300['Reference1'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
cbiz_asset_and_book_2300_merge = cbiz_asset_and_book_2300_merge.sort_index()

In [ ]:
cbiz_asset_and_book_2300_merge.index

In [ ]:
assets_group_2300.index

In [ ]:
assets_group_2300

In [ ]:
# isin() method returns a boolean array
# cbiz_asset_and_book_2300_merge.index.isin(assets_group_2300.index)

In [ ]:
# uset the boolean array as a filter in the df returns a df with matching index where array is True

cbiz_infor_2300_matched = cbiz_asset_and_book_2300_merge[cbiz_asset_and_book_2300_merge.index.isin(assets_group_2300.index)]

In [ ]:
# use the '~' with .isin method to get notin results in the returned df

missing_assets_2300 = cbiz_asset_and_book_2300_merge[~cbiz_asset_and_book_2300_merge.index.isin(assets_group_2300.index)]
missing_assets_2300

In [ ]:
missing_assets_2300.columns

In [ ]:
missing_2300__records_acquired_value = missing_assets_2300['Historical Cost'].sum()

In [ ]:
# assets_group_2300.info()

In [ ]:
asset_register_2300_cost = assets_group_2300['PostingBookBasis'].apply(lambda x: float(x)).sum()

In [ ]:
# comparing sum of missing records and total of records in FSM asset register indicates
# that there is still missing cost; next step is to explore the cost column of the matched
# records to see if there are differences in the asset register and the cbiz_load merged file

sum_asset_register_and_missing_records_cost = asset_register_2300_cost + missing_2300__records_acquired_value
sum_asset_register_and_missing_records_cost

In [ ]:
cbiz_asset_and_book_2300_merge.columns

In [ ]:
cbiz_asset_and_book_2300_merge[['Historical Cost', 'Basis', 'Accumulated Depreciation', 'Net Book Value']].sum()

In [ ]:
# assets_group_2300.info()

In [ ]:
# create a df from fsm asset group that contains the fsm assigned asset number

fsm_asset_number = assets_group_2300[['Asset', 'cbiz_index']]
fsm_asset_number

### Calculate company Nyack 2300 depreciation expense for fiscal 2021 and compare to CBIZ projections

In [ ]:
fsm_depreciation = pd.read_csv(r"C:\Users\mikra\OneDrive\Documents\Work\conversion\SIT3\november_AX2\fixed_assets\Depreciation By Asset.csv",
                              usecols=['Asset', 'CurrentPeriodDepreciation'])

In [ ]:
fsm_depreciation.columns

In [ ]:
cbiz_asset_and_book_2300_merge.head()

In [ ]:
cbiz_asset_and_book_2300_merge.info()

In [ ]:
cbiz_asset_and_book_2300_merge.columns

In [ ]:
df_depreciation_calc_2300 = cbiz_asset_and_book_2300_merge[[
    'AssetImport_x',  'Asset Display Id', 'Description', 'InServiceDate_dt',
    'Historical Cost', 'Life', 'Period Depreciation',
    'Accumulated Depreciation', 'Net Book Value', 'LifeRemaining', 'Basis',
     'Convention','LifeToDateDepreciation',
]]

In [ ]:
df_depreciation_calc_2300.tail(5)

In [ ]:
df_depreciation_calc_2300['Life'].unique()

In [ ]:
# data team question can we use dummy description
df_depreciation_calc_2300[df_depreciation_calc_2300['Description'].isnull()]

In [ ]:
# df_depreciation_calc_2300['full_dep_yr'].unique()

In [ ]:
df_depreciation_calc_2300.info()

In [ ]:
# df_depreciation_calc_2300.loc[49631]['InServiceDate_dt'] +\
# relativedelta(years=df_depreciation_calc_2300.loc[49631]['life_years'])

In [ ]:
df_depreciation_calc_2300['Life_years'] = df_depreciation_calc_2300['Life'] / 12
df_depreciation_calc_2300['Life_years'] = df_depreciation_calc_2300['Life_years'].astype('int')

In [ ]:
df_depreciation_calc_2300['full_dep_yr'] =  df_depreciation_calc_2300['Life_years']\
.apply(lambda x: dateutil.relativedelta.relativedelta(years=x))

In [ ]:
df_depreciation_calc_2300['full_dep_yr_2'] = df_depreciation_calc_2300['InServiceDate_dt'] +\
df_depreciation_calc_2300['full_dep_yr']

In [ ]:
df_depreciation_calc_2300 = df_depreciation_calc_2300.merge(fsm_asset_number, how='left',
                                            left_on='Asset Display Id', right_on="cbiz_index")

In [ ]:
df_depreciation_calc_2300.set_index('Asset Display Id', inplace=True)

In [ ]:
df_depreciation_calc_2300['Life_years'].unique()

In [ ]:
df_depreciation_calc_2300.tail(5)

In [ ]:
# test_df = df_depreciation_calc_3080.loc[[38285, 39094, 39860, 48386, 49631 ]]
# test_df.loc[48386, 'life_years'] = 2
# test_df.loc[48386, 'full_dep_yr_2'] = pd.datetime(2020, 7, 1)
# test_df

In [ ]:
balance_sheet_date = datetime(2020, 12, 31)
balance_sheet_date_2021 = datetime(2021, 12, 31)

In [ ]:
df_depreciation_calc_2300.columns

In [ ]:
df_depreciation_calc_2300.info()

In [ ]:
def dep_calc_2020(row):
    """
    This function takes a dataframe row as a parameter and
    calculates the straight line method depreciation using a half year convention.
    It adds a column for the 2020 calculated depreciation to the dataframe
    """
          
    if row['Life_years'] < 1:
        row['dep_exp_2020'] = 0
        
       
    elif row['InServiceDate_dt'].year == balance_sheet_date.year:
       
        row['dep_exp_2020'] = row['Historical Cost']\
        / row['Life_years'] * .5
    
    elif row['full_dep_yr_2'].year == balance_sheet_date.year:
        row['dep_exp_2020'] = row['Historical Cost']\
        / row['Life_years'] * .5
      
    elif (row['full_dep_yr_2'].year > balance_sheet_date.year) &\
    (row['InServiceDate_dt'].year < balance_sheet_date.year):
        
        row['dep_exp_2020'] = row['Historical Cost'] / row['Life_years']
         
    else:
        row['dep_exp_2020'] = 0
    
    return row

In [ ]:
def dep_calc_2021(row):
    
    """
    This function takes a dataframe row as a parameter and
    calculates the straight line method depreciation using a half year convention.
    It adds a column for the 2021 calculated depreciation to the dataframe
    """
          
 
    if row['Life_years'] < 1 :
        row['dep_exp_2021'] = 0
        
    elif row['InServiceDate_dt'].year == balance_sheet_date_2021.year:
       
        row['dep_exp_2021'] = row['Historical Cost']\
        / row['Life_years'] * .5
    
    elif row['full_dep_yr_2'].year == balance_sheet_date_2021.year:
        row['dep_exp_2021'] = row['Historical Cost']\
        / row['Life_years'] * .5
      
    elif (row['full_dep_yr_2'].year > balance_sheet_date_2021.year) &\
    (row['InServiceDate_dt'].year < balance_sheet_date_2021.year):
        
        row['dep_exp_2021'] = row['Historical Cost'] / row['Life_years']
         
    else:
        row['dep_exp_2021'] = 0
    
    return row
    

In [ ]:
# add a column for 2020 depreciation
df_depreciation_calc_2300 = df_depreciation_calc_2300.apply(dep_calc_2020, axis=1)

In [ ]:
# compare calculated 2020 depreciation with Current YTD depreciation column (from cbiz)
df_depreciation_calc_2300[['Period Depreciation', 'dep_exp_2020' ]].sum()

In [ ]:
# calculate 2021 depreciation and add column to df
df_depreciation_calc_2300 = df_depreciation_calc_2300.apply(dep_calc_2021, axis=1)

In [ ]:
df_depreciation_calc_2300.columns

In [ ]:
df_depreciation_calc_2300[['Period Depreciation', 'dep_exp_2020', 'dep_exp_2021' ]].sum()

In [ ]:
# filter the df for columns that have a different cbiz depreciation expense and calculated depreciation expense
df_depreciation_calc_2300[(df_depreciation_calc_2300['Period Depreciation'] != df_depreciation_calc_2300['dep_exp_2020'])]  

In [ ]:
df_depreciation_calc_2300[abs(df_depreciation_calc_2300['Period Depreciation'] - df_depreciation_calc_2300['dep_exp_2020']) > 100]

In [ ]:
# filter the df for columns that have a different depreciation expense for 2020 and 2021
df_depreciation_calc_2300[df_depreciation_calc_2300['dep_exp_2020'] != df_depreciation_calc_2300['dep_exp_2021' ]]


In [ ]:
# populate a column with 2020 YTD depreciation expense through November 2021

df_depreciation_calc_2300['depr_ytd_oct'] = df_depreciation_calc_2300['dep_exp_2021'] * (10/12)

In [ ]:
df_depreciation_calc_2300[['Period Depreciation', 'dep_exp_2020', 'dep_exp_2021', 'depr_ytd_oct' ]].sum()

In [ ]:
# cbiz_asset_and_book_2300_merge = cbiz_asset_and_book_3080_merge.merge(fsm_asset_number, how='left',
#                                             left_on='Asset ID', right_on="cbiz_index")

In [ ]:
# cbiz_asset_and_book_3080_merge[' Acquired Value* '].sum()

In [ ]:
# df_depreciation_calc_3080[' Acquired Value* '].sum()

In [ ]:
# df_depreciation_calc_2300 = df_depreciation_calc_3080.merge(fsm_asset_number, how='left',
#                                             left_on='Asset ID', right_on="cbiz_index")

In [ ]:
df_depreciation_calc_2300.head()

In [ ]:
df_depreciation_calc_2300.info()

In [ ]:
df_depreciation_calc_2300.columns

In [ ]:
df_depreciation_calc_2300.head()

In [ ]:
df_depreciation_calc_2300 = df_depreciation_calc_2300[['AssetImport_x', 'Description', 'InServiceDate_dt',
       'Historical Cost',  'Basis',  'Period Depreciation', 'LifeToDateDepreciation',
       'Net Book Value', 'Life_years','full_dep_yr_2', 'LifeRemaining', 'dep_exp_2020', 'dep_exp_2021', 'depr_ytd_oct', 'Asset' ]]

df_depreciation_calc_2300 = df_depreciation_calc_2300.merge(fsm_depreciation, how='left',
                                                           left_on='Asset', right_on='Asset')

df_depreciation_calc_2300 = df_depreciation_calc_2300[['AssetImport_x', 'Asset','Description', 'InServiceDate_dt',
       'Historical Cost',  'Basis', 'Period Depreciation', 'LifeToDateDepreciation',
       'Net Book Value', 'Life_years','full_dep_yr_2', 'LifeRemaining', 'dep_exp_2020', 'dep_exp_2021', 'depr_ytd_oct', 'CurrentPeriodDepreciation'  ]]

In [ ]:
df_depreciation_calc_2300.tail()

In [ ]:
# df_depreciation_calc_2300.to_csv(
#     r'C:\Users\mikra\OneDrive\Documents\Work\conversion\SIT3\november_AX2\fixed_assets\nyack\output_2300\sit3_depreciation_2300.csv'
# )